Library Imports

In [4]:
import pandas as pd

import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

Data Import

In [36]:
df_full = pd.read_csv(r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Data\collatedData17-20.csv')
df = df_full


Pre-Processing

Filter dataframe to player data from just the years 17/18 and 20/21.

Filter this further to Just those in the forward positions

In [6]:
roles=['FW','FWDF','FWMF','MFFW']
df = df[((df['y2017-18_Pos'].notnull()) & (df['y2020-21_Pos'].isin(roles)))]
df

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
6,Adam Lallana\99813635,258,eng ENG,MFFW,Brighton,32,1988,30,16,1596,...,6.0,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3
9,Ademola Lookman\7c104bb7,281,eng ENG,FWMF,Fulham,22,1997,34,31,2747,...,0.0,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0
15,Aleksandar MitroviÃÂ\3925dbd6,329,rs SRB,FW,Fulham,25,1994,27,13,1402,...,1.0,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8
16,Alex Iwobi\6ca5ec4b,216,ng NGA,FWDF,Everton,24,1996,30,17,1542,...,29.0,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0
18,Alex Oxlade-Chamberlain\38c7feef,364,eng ENG,FWMF,Liverpool,26,1993,13,2,253,...,40.0,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,Theo Walcott\1ecb65be,499,eng ENG,FW,Everton,31,1989,1,0,13,...,40.0,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6
525,Wilfried Zaha\b2bc3b1f,526,ci CIV,FW,Crystal Palace,27,1992,30,29,2612,...,85.0,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9
530,Willian\8b9ebd03,516,br BRA,FWMF,Arsenal,31,1988,25,16,1406,...,51.0,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4
533,Xherdan Shaqiri\6421ec64,435,ch SUI,MFFW,Liverpool,28,1991,14,5,556,...,101.0,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8


Label the Rows.
At the moment it is very simple: 8 goals or over is a 1.

This is a first attempt, so is simple by design

In [7]:
pd.options.mode.chained_assignment = None  # default='warn'
def calc_label(row):
    if row['y2020-21_Gls'] < 8:
        value = 0
    else:
        value = 1
    return value

df['label'] = df.apply(calc_label, axis=1)
df['label'].value_counts()

0    55
1    23
Name: label, dtype: int64

Remove data columns for the 20/21 season, so that the models can train on the 17/18 season data

In [8]:
df = df[df.columns[-150:]]
df

,y2017-18_Nation,y2017-18_Pos,y2017-18_Squad,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
6,eng ENG,MFFW,Liverpool,29.0,1988.0,12.0,1.0,308.0,3.4,0.0,...,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3,0
9,eng ENG,FWMF,Everton,19.0,1997.0,7.0,1.0,159.0,1.8,0.0,...,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0,0
15,rs SRB,FW,Newcastle Utd,22.0,1994.0,6.0,0.0,95.0,1.1,1.0,...,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8,0
16,ng NGA,FWMF,Arsenal,21.0,1996.0,26.0,22.0,1830.0,20.3,3.0,...,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0,0
18,eng ENG,MFFW,Liverpool,23.0,1993.0,32.0,14.0,1493.0,16.6,3.0,...,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,eng ENG,FW,Everton,28.0,1989.0,14.0,13.0,1152.0,12.8,3.0,...,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6,0
525,ci CIV,FWMF,Crystal Palace,24.0,1992.0,29.0,28.0,2549.0,28.3,9.0,...,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9,1
530,br BRA,FWMF,Chelsea,28.0,1988.0,36.0,20.0,1890.0,21.0,6.0,...,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4,0
533,ch SUI,FWMF,Stoke City,25.0,1991.0,36.0,36.0,3039.0,33.8,8.0,...,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8,0


Remove unneccessary fields from the 17/28 data

In [9]:
df = df.drop(['y2017-18_Nation',
                                'y2017-18_Pos',
                                'y2017-18_Squad']
                                ,1)
df

,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,y2017-18_Ast,y2017-18_G-PK,y2017-18_PK,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
6,29.0,1988.0,12.0,1.0,308.0,3.4,0.0,0.0,0.0,0.0,...,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3,0
9,19.0,1997.0,7.0,1.0,159.0,1.8,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0,0
15,22.0,1994.0,6.0,0.0,95.0,1.1,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8,0
16,21.0,1996.0,26.0,22.0,1830.0,20.3,3.0,5.0,3.0,0.0,...,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0,0
18,23.0,1993.0,32.0,14.0,1493.0,16.6,3.0,7.0,3.0,0.0,...,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,28.0,1989.0,14.0,13.0,1152.0,12.8,3.0,3.0,3.0,0.0,...,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6,0
525,24.0,1992.0,29.0,28.0,2549.0,28.3,9.0,3.0,9.0,0.0,...,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9,1
530,28.0,1988.0,36.0,20.0,1890.0,21.0,6.0,7.0,5.0,1.0,...,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4,0
533,25.0,1991.0,36.0,36.0,3039.0,33.8,8.0,7.0,8.0,0.0,...,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8,0


Normalise Data using MinMax Scaling for each column.

For future development stages it may be worth leaving some columns out from this stage, aswell as adding extra columns based on the data available, but this is a first run, and so the aim is to keep things simple.

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

df_normalized.dropna()
df_normalized

,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,y2017-18_Ast,y2017-18_G-PK,y2017-18_PK,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
0,0.928571,0.142857,0.297297,0.027027,0.092341,0.091667,0.00000,0.000000,0.000000,0.0,...,0.096154,0.047619,0.000000,0.0,0.0,0.125000,0.011494,0.022989,0.333,0.0
1,0.214286,0.785714,0.162162,0.027027,0.046325,0.047222,0.00000,0.000000,0.000000,0.0,...,0.038462,0.031746,0.000000,0.0,0.0,0.062500,0.003831,0.003831,0.500,0.0
2,0.428571,0.571429,0.135135,0.000000,0.026560,0.027778,0.03125,0.000000,0.032258,0.0,...,0.019231,0.000000,0.000000,0.0,0.0,0.035156,0.015326,0.034483,0.308,0.0
3,0.357143,0.714286,0.675676,0.594595,0.562384,0.561111,0.09375,0.454545,0.096774,0.0,...,0.230769,0.333333,0.000000,0.0,0.0,0.582031,0.026820,0.080460,0.250,0.0
4,0.500000,0.500000,0.837838,0.378378,0.458308,0.458333,0.09375,0.636364,0.096774,0.0,...,0.384615,0.365079,0.000000,0.0,0.0,0.597656,0.061303,0.118774,0.340,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.857143,0.214286,0.351351,0.351351,0.352996,0.352778,0.09375,0.272727,0.096774,0.0,...,0.269231,0.190476,0.000000,0.0,0.0,0.390625,0.030651,0.111111,0.216,0.0
74,0.571429,0.428571,0.756757,0.756757,0.784435,0.783333,0.28125,0.272727,0.290323,0.0,...,0.365385,0.380952,0.666667,0.0,0.0,0.726562,0.007663,0.187739,0.039,1.0
75,0.857143,0.142857,0.945946,0.540541,0.580914,0.580556,0.18750,0.636364,0.161290,0.2,...,0.230769,0.380952,0.166667,0.0,0.0,0.554688,0.042146,0.088123,0.324,0.0
76,0.642857,0.357143,0.945946,0.972973,0.935763,0.936111,0.25000,0.636364,0.258065,0.0,...,0.576923,0.253968,0.000000,0.0,0.0,0.816406,0.038314,0.122605,0.238,0.0


Creating Training / Testing Datasets from the normalised data

In [30]:
from sklearn.model_selection import train_test_split
y = df[['label']]
X = df_normalized.drop(['label'],1)


print('Length of x: ',len(X))
print('Length of y: ',len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

X_test.fillna(X_train.mean(), inplace=True)
X_train.fillna(X_train.mean(), inplace=True)


print('Length of X_train: ',len(X_train))
print('Length of X_test: ',len(X_test))
print('Length of y_train: ',len(y_train))
print('Length of y_test: ',len(y_test))

Length of x:  78
Length of y:  78
Length of X_train:  39
Length of X_test:  39
Length of y_train:  39
Length of y_test:  39


Model Development

SVM Classifier

In [12]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[23  5]
 [ 7  4]]
              precision    recall  f1-score   support

           0       0.77      0.82      0.79        28
           1       0.44      0.36      0.40        11

    accuracy                           0.69        39
   macro avg       0.61      0.59      0.60        39
weighted avg       0.68      0.69      0.68        39



c:\Users\jacks\env\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\SVM01.pkl'
pickle.dump(svclassifier, open(fn, 'wb'))

RandomForest Classification

In [15]:
RFclassifier = RandomForestClassifier()
RFclassifier.fit(X_train, y_train)

y_pred = RFclassifier.predict(X_test)


print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[[27  1]
 [ 6  5]]
              precision    recall  f1-score   support

           0       0.82      0.96      0.89        28
           1       0.83      0.45      0.59        11

    accuracy                           0.82        39
   macro avg       0.83      0.71      0.74        39
weighted avg       0.82      0.82      0.80        39



In [19]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\RMF01.pkl'
pickle.dump(RFclassifier, open(fn, 'wb'))

Naive Bayes

In [16]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[27  1]
 [ 8  3]]
              precision    recall  f1-score   support

           0       0.77      0.96      0.86        28
           1       0.75      0.27      0.40        11

    accuracy                           0.77        39
   macro avg       0.76      0.62      0.63        39
weighted avg       0.77      0.77      0.73        39



c:\Users\jacks\env\lib\site-packages\sklearn\naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\GNB01.pkl'
pickle.dump(gnb, open(fn, 'wb'))

K-NearestNeighbour

In [17]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[24  4]
 [10  1]]
              precision    recall  f1-score   support

           0       0.71      0.86      0.77        28
           1       0.20      0.09      0.13        11

    accuracy                           0.64        39
   macro avg       0.45      0.47      0.45        39
weighted avg       0.56      0.64      0.59        39



c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [24]:
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[27  1]
 [ 8  3]]
              precision    recall  f1-score   support

           0       0.77      0.96      0.86        28
           1       0.75      0.27      0.40        11

    accuracy                           0.77        39
   macro avg       0.76      0.62      0.63        39
weighted avg       0.77      0.77      0.73        39



c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [25]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\KNN01.pkl'
pickle.dump(neigh, open(fn, 'wb'))

Analysis

In [61]:
RFclassifier = RandomForestClassifier()
RFclassifier.fit(X_train, y_train)

y_pred = RFclassifier.predict(X_test)


out = y_test
out['pred'] = y_pred

def createOutputDF(df):
    outputmatrix=[]
    for i, row in df.iterrows():
        outputmatrix.append(df_full.loc[[i]].values.flatten().tolist())
    df_output = pd.DataFrame(outputmatrix,columns=df_full.columns)
    return df_output

df_TP = out[((out['label']==1) & (out['pred']==1))]
df_FP = out[((out['label']==0) & (out['pred']==1))]
df_TN = out[((out['label']==0) & (out['pred']==0))]
df_FN = out[((out['label']==1) & (out['pred']==0))]
df_TP = createOutputDF(df_TP)
df_FP = createOutputDF(df_FP)
df_TN = createOutputDF(df_TN)
df_FN = createOutputDF(df_FN)


c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


True Positives

In [62]:
df_TP

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Raheem Sterling\b400bde0,456,eng ENG,FW,Manchester City,25,1994,31,28,2536,...,51.0,17.0,20.0,6.0,1.0,0.0,186.0,16.0,35.0,31.4
1,Gabriel Jesus\b66315ae,224,br BRA,FW,Manchester City,23,1997,29,22,2063,...,8.0,13.0,17.0,0.0,0.0,0.0,79.0,21.0,61.0,25.6
2,Chris Wood\4e9a0555,521,nz NZL,FW,Burnley,28,1991,33,32,2741,...,7.0,4.0,0.0,0.0,0.0,0.0,78.0,99.0,161.0,38.1
3,Riyad Mahrez\892d5bb1,290,dz ALG,FW,Manchester City,29,1991,27,23,1949,...,73.0,32.0,26.0,0.0,0.0,0.0,256.0,23.0,57.0,28.8
4,Callum Wilson\c596fcb0,519,eng ENG,FW,Newcastle Utd,28,1992,26,23,2084,...,13.0,5.0,10.0,0.0,0.0,0.0,105.0,35.0,115.0,23.3
5,Mohamed Salah\e342ad68,422,eg EGY,FW,Liverpool,28,1992,37,34,3078,...,50.0,13.0,13.0,1.0,0.0,0.0,218.0,19.0,58.0,24.7


False Positives

In [63]:
df_FP

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Richarlison\fa031b34,398,br BRA,FW,Everton,23,1997,34,33,2861,...,49.0,35.0,37.0,2.0,0.0,0.0,212.0,114.0,167.0,40.6
1,Dwight Gayle\e58a03c6,171,eng ENG,FWMF,Newcastle Utd,29,1990,18,4,514,...,8.0,8.0,8.0,0.0,0.0,0.0,78.0,65.0,131.0,33.2
2,Ayoze PÃÂ©rez\819aa8e7,373,es ESP,MFFW,Leicester City,27,1993,25,15,1331,...,27.0,32.0,21.0,0.0,0.0,1.0,198.0,51.0,124.0,29.1


True Negatives

In [64]:
df_TN

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Ruben Loftus-Cheek\e97fd090,276,eng ENG,FW,Chelsea,24,1996,1,1,60,...,12.0,26.0,23.0,1.0,0.0,0.0,183.0,32.0,64.0,33.3
1,Oliver Burke\f631c17f,74,sct SCO,FW,Sheffield Utd,23,1997,25,14,1269,...,8.0,7.0,5.0,0.0,0.0,0.0,24.0,8.0,8.0,50.0
2,Tammy Abraham\f586779e,2,eng ENG,FW,Chelsea,22,1997,22,12,1040,...,14.0,5.0,6.0,0.0,0.0,0.0,66.0,53.0,87.0,37.9
3,Michael Obafemi\caa1a7f0,356,ie IRL,FW,Southampton,20,2000,4,0,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Eddie Nketiah\a53649b7,348,eng ENG,FW,Arsenal,21,1999,17,4,423,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0
5,Michy Batshuayi\2973d8ff,50,be BEL,FW,Crystal Palace,26,1993,18,7,728,...,0.0,1.0,4.0,0.0,0.0,1.0,17.0,8.0,17.0,32.0
6,Nathan Redmond\ab651565,392,eng ENG,MFFW,Southampton,26,1994,29,17,1738,...,48.0,20.0,16.0,0.0,0.0,0.0,160.0,12.0,32.0,27.3
7,Olivier Giroud\16ceb862,178,fr FRA,FW,Chelsea,33,1986,17,8,748,...,2.0,2.0,5.0,0.0,0.0,0.0,32.0,53.0,30.0,63.9
8,Jordan Ayew\da052c14,35,gh GHA,FWMF,Crystal Palace,28,1991,33,23,2096,...,37.0,35.0,63.0,0.0,0.0,0.0,198.0,67.0,155.0,30.2
9,Alex Oxlade-Chamberlain\38c7feef,364,eng ENG,FWMF,Liverpool,26,1993,13,2,253,...,40.0,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0


False Negatives

In [65]:
df_FN

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Harvey Barnes\3ea50f67,47,eng ENG,FWMF,Leicester City,22,1997,25,22,1945,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0
1,Danny Ings\07802f7f,214,eng ENG,FW,Southampton,28,1992,29,26,2173,...,2.0,1.0,3.0,0.0,0.0,0.0,14.0,6.0,9.0,40.0
